# 朴素贝叶斯实践——文本分类

之所以称之为朴素贝叶斯，是因为它假设每个输入变量都是独立的。这是一个强硬的假设，实际情况并不一定，但是这项技术对于绝大部分的复杂问题仍然的有效的。

朴素贝叶斯分类常用于文本分类，尤其是对于英文等语言来说，分类效果很好。它常用于垃圾文本过滤、情感预测、推荐系统等。

# 加载停止词

所谓停止词，就是不能帮助区分文档类型的词

In [3]:
# 加载停止词
stop_words = [line.strip() for line in open('./stop/stopword.txt', 'r', encoding='utf-8').readlines()]

# 分词&打标

中文一般使用jieba进行分词，英文一般使用nltk进行分词 

分词与打标的结果：
[ 文件1的分词，中间以空格隔开]：label
[ 文件2的分词，中间以空格隔开]：label
...
[ 文件n的分词，中间以空格隔开]：label

In [10]:
# 分词与打标

import os 
import jieba 

def cut_words(file_path):
    words = []
    with open(file_path, 'r', encoding='gb18030') as file:
        text = file.read()
        words = [word for word in jieba.cut(text)]
    
    return ' '.join(words)

def loadfile(file_dir, label):
    """
    将路径下的所有文件加载
    :param file_dir: 保存txt文件目录
    :param label: 文档标签
    :return: 分词后的文档列表和标签
    """
    file_list = os.listdir(file_dir)
    words_list = []
    labels_list = []
    for file in file_list:
        file_path = file_dir + '/' + file
        words_list.append(cut_words(file_path))
        labels_list.append(label)                                                                                                                 
    return words_list, labels_list

# 加载训练数据
train_words_list1, train_labels1 = loadfile('./train/女性', '女性')
train_words_list2, train_labels2 = loadfile('./train/体育', '体育')
train_words_list3, train_labels3 = loadfile('./train/文学', '文学')
train_words_list4, train_labels4 = loadfile('./train/校园', '校园')

train_words_list = train_words_list1 + train_words_list2 + train_words_list3 + train_words_list4    
train_labels = train_labels1 + train_labels2 + train_labels3 + train_labels4

# 加载测试数据
test_words_list1, test_labels1 = loadfile('./test/女性', '女性')
test_words_list2, test_labels2 = loadfile('./test/体育', '体育')
test_words_list3, test_labels3 = loadfile('./test/文学', '文学')
test_words_list4, test_labels4 = loadfile('./test/校园', '校园')

test_words_list = test_words_list1 + test_words_list2 + test_words_list3 + test_words_list4
test_labels = test_labels1 + test_labels2 + test_labels3 + test_labels4

# 计算单词权重

使用tf-idf计算单词权重，max_df=0.5表示单词在文档中出现的频率超过50%的文档将被忽略 

TF：单词在一个文件出现的频率

$$
TF = \frac{单词在文件中出现的次数}{文件中单词总数}
$$

IDF：单词在所有文件中出现的频率
$$
IDF = \log(\frac{文件总数}{包含单词的文件数 + 1})
$$

TF-IDF：TF与IDF的乘积。其实就是综合考虑TF和IDF，TF高的单词，可能是没啥区分度的单词，比如'我'，'的'，'是'等。 IDF高的单词，说明就是区分度很高的。
$$
TF-IDF = TF \times IDF
$$

In [12]:
# 计算单词权重

from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(stop_words=stop_words, max_df=0.5)
train_features = tf.fit_transform(train_words_list)
test_features = tf.transform(test_words_list)

# 贝叶斯分类器

In [13]:
# 多项式贝叶斯分类器
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB(alpha=0.001).fit(train_features, train_labels)

# 预测&计算准确率

In [14]:
# 预测&计算准确率
from sklearn.metrics import accuracy_score

predicted_labels = clf.predict(test_features)
print(accuracy_score(test_labels, predicted_labels))

0.91
